# Neural Networks

Neural networks can be constructedd using the `torch.nn` package.

Now that you had a glimpse of `autograd`, `nn` depends on `autograd` to define models and differentiate them. An nn.Module contains layers, and a method `forward(input)` that returns the output.

A typical training procedure for a neural network is as follow: 

- Define the neural network that has some learnable parameters (or weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back into the network's parameters
- Update the weights of the network, typically usind a simple update rule: `weight = weight - learning_rate * gradient` 

## Define the network

Let's define this network:

In [8]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [10]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


You just have to define the `forward` function and the `backward` function (where gradient are computed) is automatically defined for you using `autograd`. You can use any of the Tensor operations in the `forward` function.

The learnable parameters of a model are returned by `net.parameters()`

The learnable parameters of a model are returned by `net.parameters()` 

In [13]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight 

10
torch.Size([6, 1, 3, 3])


Let's try a random 32x32 input. Note: expected input size of this net (LeNet) is 32x32. To use this net on the MNIST dataset, please resize the images from the dastaset to 32x32.

In [14]:
input = torch.rand(1,1,32,32)
out = net(input)
print(out)

tensor([[-0.0668,  0.0813, -0.0215, -0.0346, -0.0046,  0.0220,  0.1251,  0.0633,
          0.0558, -0.0721]], grad_fn=<AddmmBackward>)


In [15]:
net.zero_grad()
out.backward(torch.rand(1,10))

## Loss Function

A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

There are several different loss functions under the nn package. A simple loss is: `nn.MSELoss` which computes the mean-squared error between the input and the target.

For example: 

In [17]:
output = net(input)
target = torch.randn(10) # a dummy target, for example
target = target.view(1,-1) # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.8914, grad_fn=<MseLossBackward>)


So, when we call `loss.backward()`, the whole graph is differentiated w.r.t the loss, and all Tensors in the graph that has `requires_grad=True` will have their `.grad` Tensor accumulated with their gradient. 

For illustration, let us follow a few steps backward:

In [19]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU